# 数据绑定

`HTree` 是一个类，用来进行数据绑定。它的实例化对象是一个层次化的树形结构，每个节点都是一个 `HTree`，`HTree` 对象的属性可以通过点号访问，也可以通过索引访问。


## 将Python中原生数据映射到`HTree`中

In [1]:
from spdm.core.htree import HTree, List, Dict

class Data(Dict):
    pass

### 传递一个普通的字典结构给Data
data = Data(
    {
        "name": "Alice",
        "age": 25,
        "hobbies": ["reading", "painting", "yoga"],
        "address": [
            {"street": "123 Main St", "city": "Anytown", "state": "CA", "zip": "12345"},
            {
                "street": "456 Oak St",
                "city": "Othertown",
                "state": "NY",
                "zip": "67890",
            },
            {"street": "789 Elm St", "city": "Somewhere", "state": "CO", "zip": "24680"},
        ],
    },
    # _entry=open_entry(....)
)

In [2]:
## 访问字典中的数据
data.get("name")

'Alice'

In [3]:
data.get("address")

[{'street': '123 Main St', 'city': 'Anytown', 'state': 'CA', 'zip': '12345'},
 {'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'},
 {'street': '789 Elm St', 'city': 'Somewhere', 'state': 'CO', 'zip': '24680'}]

In [4]:
### 但是address不支持直接切片访问。
data.get("address[1]")

{'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'}

In [5]:
### address只能以整体拿回来，然后再切片访问
data.get("address")[1]

{'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'}

## `sp_tree` 装饰器
`sp_tree` 装饰器的作用是根据 `class` 中定义的 `type_hint` 生成 `property`，并于数据源对应 `key` 的 `value` 绑定，且在访问 `property` 时自动根据 `type_hint` 进行类型转换。

In [6]:
from spdm.core.sp_tree import sp_property, SpTree
from spdm.core.htree import List
import pprint


### sp_tree装饰器支持在Data类中声明字典中的每个元素key及其value的数据类型，这样就可以直接访问字典中的元素。


class Data(SpTree):
    name: str
    age: int
    hobbies: List[str]
    address: List[Dict]


data = Data(
    {
        "name": "Alice",
        "age": "25",
        "hobbies": ["reading", "painting", "yoga"],
        "address": [
            {"street": "123 Main St", "city": "Anytown", "state": "CA", "zip": "12345"},
            {
                "street": "456 Oak St",
                "city": "Othertown",
                "state": "NY",
                "zip": "67890",
            },
            {
                "street": "789 Elm St",
                "city": "Somewhere",
                "state": "CO",
                "zip": "24680",
            },
        ],
    },
    # _entry=open_entry(....)
)

In [7]:
## 发现声明过的元素其颜色都是蓝色的
data.name

'Alice'

In [8]:
### 除了声明元素的key值，同时可以强制转化原有数据的数据类型
type(data.age)

int

In [9]:
### 
data.address

<spdm.core.htree.List[spdm.core.htree.Dict] at 0x7f4abbc48610>

In [10]:
### 支持对LIST元素的直接切片访问，而不需要拿回整个字典
data.address[0].__getstate__()

{'street': None,
 'city': None,
 'state': None,
 'zip': None,
 '$type': 'spdm.core.htree.Dict',
 '$entry': {'_cache': Ellipsis, '_path': address/0}}

## List[HTree] 结构体数组

In [11]:
### 增加Address类，定义address的数据类型


class Addrees(SpTree):
    street: str
    city: str
    state: str
    zip: int


class Data(SpTree):

    name: str
    age: int
    hobbies: List[str]
    address: List[Addrees]


data = Data(
    {
        "name": "Alice",
        "age": "25",
        "hobbies": ["reading", "painting", "yoga"],
        "address": [
            {"street": "123 Main St", "city": "Anytown", "state": "CA", "zip": "12345"},
            {
                "street": "456 Oak St",
                "city": "Othertown",
                "state": "NY",
                "zip": "67890",
            },
            {
                "street": "789 Elm St",
                "city": "Somewhere",
                "state": "CO",
                "zip": "24680",
            },
        ],
    },
    # _entry=open_entry(....)
)

In [12]:
### zip的颜色变成了蓝色，因为在Addrees类中声明了zip
data.address[0].zip

12345

In [13]:
data.address[1].street

'456 Oak St'

In [14]:
data.address[0]._cache

{'street': '123 Main St', 'city': 'Anytown', 'state': 'CA', 'zip': 12345}

## 访问属性

In [15]:
class Addrees(SpTree):
    street: str
    city: str
    state: str
    zip: int
    building: str = "#12345"  # default value
    neighbour: str


class Data(SpTree):

    name: str
    age: int
    hobbies: List[str]
    address: List[Addrees]


data = Data(
    {
        "name": "Alice",
        "age": 25,
        "hobbies": ["reading", "painting", "yoga"],
        "address": [
            {"street": "123 Main St", "city": "Anytown", "state": "CA", "zip": "12345"},
            {
                "street": "456 Oak St",
                "city": "Othertown",
                "state": "NY",
                "zip": "67890",
            },
            {
                "street": "789 Elm St",
                "city": "Somewhere",
                "state": "CO",
                "zip": "24680",
            },
        ],
    },
    # _entry=open_entry(....)
)

In [16]:
data.address[0].building

'#12345'

In [17]:
data.address[0].neighbour="zhangshan"

In [18]:
### 应用才会生效
data.address[1].building

'#12345'

In [19]:
data.address[0].neighbour

'zhangshan'

In [20]:
data.address[1].neighbour = "lisi"

In [21]:
data.address[0]._cache

{'street': '123 Main St',
 'city': 'Anytown',
 'state': 'CA',
 'zip': '12345',
 'building': '#12345',
 'neighbour': 'zhangshan'}

In [22]:
data.address[1].neighbour

'lisi'

### 增加函数sp_property，定义函数

In [23]:
class House(SpTree):

    level: int = 4
    length: float
    width: float

    @sp_property(units="m^2")
    def area(self) -> float:
        return self.width * self.length


class Addrees(SpTree):
    street: str
    city: str
    state: str
    zip: int
    building: str = "#12345"
    neighbour: str
    house: House = sp_property(
        label="big house", default_value={"level": 2, "area": 1000}
    )


class Data(SpTree):

    name: str
    age: int
    hobbies: List[str]
    address: List[Addrees]


data = Data(
    {
        "name": "Alice",
        "age": 25,
        "hobbies": ["reading", "painting", "yoga"],
        "address": [
            {"street": "123 Main St", "city": "Anytown", "state": "CA", "zip": "12345"},
            {
                "street": "456 Oak St",
                "city": "Othertown",
                "state": "NY",
                "zip": "67890",
                "house": {"length": 5, "width": 10},
            },
            {
                "street": "789 Elm St",
                "city": "Somewhere",
                "state": "CO",
                "zip": "24680",
            },
        ],
    },
    # _entry=open_entry(....)
)

In [24]:
house=data.address[1].house

In [25]:
house.level

4

In [26]:
house.length

5.0

In [27]:
house.width

10.0

In [28]:
house.area

50.0

In [29]:
House.area.metadata.get("units")

'm^2'

In [30]:
data.get("address/3/street", "I don't know")

"I don't know"

In [31]:
for address in data.address:
    print(address.street)

123 Main St
456 Oak St
789 Elm St


In [32]:
data.address[0].street = "456 Main St"

In [33]:
data.address[0].street

'456 Main St'